## Variant data selection and preprocessing (__Humsavar__)

In [36]:
import json
import os
import pandas as pd
import re
import requests
import time
import VEP
from Bio.Data import IUPACData
from datetime import datetime
from pathlib import Path

In [3]:
def parse_humsavar(file):
    """
    Parse humsavar.txt file content and extract the variant table into a pandas DataFrame.
    
    Parameters:
    file (str): humsavar.txt
    
    Returns:
    pandas.DataFrame: DataFrame containing the variant data with columns:
        - Main gene name
        - Swiss-Prot AC
        - FTId
        - AA change
        - Variant category
        - dbSNP
        - Disease name
    """
    # split content to lines
    lines = file.split('\n')

    # find start of the table
    table_start = None
    for i, line in enumerate(lines):
        if line.startswith('Main   '):
            table_start = i
            break
    
    if table_start is None:
        raise ValueError("Could not find the table header in the file")
    
    # skip header and underscores lines
    data_start = table_start + 2
    
    data = []
    for line in lines[data_start:]:
        if not line.strip():
            continue
            
        # split line into fields while preserving whitespace
        fields = re.split(r'\s{2,}', line.strip())
        
        # check all required fields have been correctly added
        if len(fields) >= 7:
            data.append({
                'Main gene name': fields[0],
                'Swiss-Prot AC': fields[1],
                'FTId': fields[2],
                'AA change': fields[3],
                'Variant category': fields[4],
                'dbSNP': fields[5],
                'Disease name': fields[6] if fields[6] != '-' else None
            })

    df = pd.DataFrame(data)
    return df

In [4]:
def read_and_parse_humsavar(file_path):
    """
    Read humsavar.txt file and parse it into a DataFrame.
    
    Parameters:
    file_path (str): Path to humsavar.txt
    
    Returns:
    pandas.DataFrame: DataFrame containing the data
    """
    with open(file_path, 'r') as file:
        content = file.read()
    return parse_humsavar(content)

In [5]:
file = '../data/humsavar/humsavar_202501.txt'
data25 = read_and_parse_humsavar(file)
data25['Source']='2025'
data25.head()

,Main gene name,Swiss-Prot AC,FTId,AA change,Variant category,dbSNP,Disease name,Source
0,A1BG,P04217,VAR_018369,p.His52Arg,LB/B,rs893184,None,2025
1,A1BG,P04217,VAR_018370,p.His395Arg,LB/B,rs2241788,None,2025
2,A1CF,Q9NQ94,VAR_052201,p.Val555Met,LB/B,rs9073,None,2025
3,A1CF,Q9NQ94,VAR_059821,p.Ala558Ser,LB/B,rs11817448,None,2025
4,A2M,P01023,VAR_000012,p.Arg704His,LB/B,rs1800434,None,2025


Read and process Humsavar files for 2025 and 2021.

Source column is added to track the dataset origin and to distinguish variants after merging.

In [6]:
file = '../data/humsavar/humsavar_202102.txt'
data21 = read_and_parse_humsavar(file)
data21['Source']='2021'
data21.head()

,Main gene name,Swiss-Prot AC,FTId,AA change,Variant category,dbSNP,Disease name,Source
0,A1BG,P04217,VAR_018369,p.His52Arg,LB/B,rs893184,None,2021
1,A1BG,P04217,VAR_018370,p.His395Arg,LB/B,rs2241788,None,2021
2,A1CF,Q9NQ94,VAR_052201,p.Val555Met,LB/B,rs9073,None,2021
3,A1CF,Q9NQ94,VAR_059821,p.Ala558Ser,LB/B,rs11817448,None,2021
4,A2M,P01023,VAR_000012,p.Arg704His,LB/B,rs1800434,None,2021


Strip whitespaces from columns.

In [7]:
compare_columns=['Main gene name', 'Swiss-Prot AC', 'AA change']
data25[compare_columns] = data25[compare_columns].apply(lambda x: x.str.strip())
data21[compare_columns] = data21[compare_columns].apply(lambda x: x.str.strip())

Remove duplicate entries (for both 2025 and 2021).

Many of the variants are already included in 2025, and as our goal is to keep the dataset with the latest information, we remove possible duplicates.

In [8]:
print("Before removing duplicates: ", len(data25))
data25_1 = data25.drop_duplicates(subset=compare_columns).reset_index(drop=True)
print("After removing duplicates: ", len(data25_1))

Before removing duplicates:  83697
After removing duplicates:  82585


In [9]:
print("Before removing duplicates: ", len(data21))
data21_1 = data21.drop_duplicates(subset=compare_columns).reset_index(drop=True)
print("After removing duplicates: ", len(data21_1))

Before removing duplicates:  79192
After removing duplicates:  78192


In [10]:
# This is the difference in unique variants between 2025 and 2021.
len(data25_1) - len(data21_1)

4393

In [11]:
# how many rows are in 2025 but not in 2021 based on compare_columns
diff_data = data25_1[~data25_1[compare_columns].apply(tuple, axis=1).isin(data21_1[compare_columns].apply(tuple, axis=1))]
print(f"Rows in 2025 but not in 2021: {len(diff_data)}")

Rows in 2025 but not in 2021: 6047


Clinical significance distribution of 2025 variants not in 2021.

In [12]:
diff_data['Variant category'].value_counts()

Variant category
LP/P    2291
US      2222
LB/B    1534
Name: count, dtype: int64

We keep only LP/P (Likely Pathogenic/Pathogenic) and LB/B (Likely Benign/Benign) variants. With this we aim to remove VUS.

In [13]:
data25_2 = diff_data[diff_data['Variant category'].isin(['LP/P','LB/B'])].reset_index(drop=True)

In [14]:
data25_2['Variant category'].value_counts()

Variant category
LP/P    2291
LB/B    1534
Name: count, dtype: int64

In [15]:
# how many genes we have
data25_2['Main gene name'].nunique()

1258

In [16]:
data25_2['Swiss-Prot AC'].nunique()

1258

In [17]:
data25_2.head()

,Main gene name,Swiss-Prot AC,FTId,AA change,Variant category,dbSNP,Disease name,Source
0,AARS1,P49588,VAR_089576,p.Arg326Trp,LP/P,-,"Charcot-Marie-Tooth disease, axonal, 2N (CMT2N...",2025
1,AARS1,P49588,VAR_089577,p.Thr606Ile,LP/P,-,"Leukoencephalopathy, hereditary diffuse, with ...",2025
2,AARS1,P49588,VAR_089578,p.Ser698Phe,LP/P,-,"Charcot-Marie-Tooth disease, axonal, 2N (CMT2N...",2025
3,ABCA4,P78363,VAR_084908,p.Asp1102Tyr,LB/B,rs138641544,None,2025
4,ABCA4,P78363,VAR_084916,p.Gly1203Asp,LP/P,-,Stargardt disease 1 (STGD1) [MIM:248200],2025


Exclude entries where the gene name corresponds to a dash

In [18]:
data25_2 = data25_2[data25_2['Main gene name']!='-'].reset_index(drop=True)
len(data25_2)

3824

Check if there are accession numbers that do not have the expected length of 6.

In [19]:
[uni for uni in data25_2['Swiss-Prot AC'].unique() if len(uni)!= 6]

[]

As with ClinVar dataset, we convert Aa changes to 1 letter notation.

In [20]:
def convert_to_one_letter(aa_change):
    prefix, from_aa, position, to_aa = (
        aa_change[0:2],  # e.g., 'p.'
        aa_change[2:5],  # e.g., 'Arg'
        ''.join(filter(str.isdigit, aa_change)),  # e.g., '326'
        aa_change[-3:]  # e.g., 'Trp'
    )

    # convert to 1 letter code
    from_aa_one = IUPACData.protein_letters_3to1.get(from_aa, '?')
    to_aa_one = IUPACData.protein_letters_3to1.get(to_aa, '?')
    return f"{from_aa_one}{position}{to_aa_one}"

In [21]:
data25_2['Variant'] = data25_2['AA change'].apply(convert_to_one_letter)

And we assign binary clinical significance.

In [22]:
data25_2.loc[data25_2['Variant category']=='LB/B','BinaryClinicalSignificance'] = 'B'
data25_2.loc[data25_2['Variant category']=='LP/P','BinaryClinicalSignificance'] = 'P'

In [23]:
data25_2.BinaryClinicalSignificance.value_counts()

BinaryClinicalSignificance
P    2291
B    1533
Name: count, dtype: int64

In [24]:
data25_2['Main gene name'].nunique()

1257

Filter duplicated dbSNP entries.

In [25]:
data25_2[data25_2.duplicated(subset=['dbSNP'], keep=False)].dbSNP.unique()

array(['-', 'rs1645264815', 'rs2153228682', 'rs1705222655', 'rs752450983',
       'rs1599011050', 'rs1949512456', 'rs28642966', 'rs201552310',
       'rs200005406', 'rs2072648', 'rs1057517926', 'rs1757708758',
       'rs77834747', 'rs381427', 'rs421016', 'rs121908310', 'rs77933015',
       'rs121908308', 'rs782199122', 'rs1191455921', 'rs2071312',
       'rs104894264', 'rs7480563', 'rs7126405', 'rs2293232', 'rs2246901',
       'rs200291894', 'rs571714796', 'rs7255187', 'rs1684813071',
       'rs1554297905', 'rs1838076782', 'rs141269120', 'rs1057149',
       'rs1385657144', 'rs1965499910'], dtype=object)

In [26]:
# use first output as example
data25_2[data25_2.dbSNP=='rs1645264815']

,Main gene name,Swiss-Prot AC,FTId,AA change,Variant category,dbSNP,Disease name,Source,Variant,BinaryClinicalSignificance
119,AGO1,Q9UL18,VAR_088408,p.Leu190Pro,LP/P,rs1645264815,Neurodevelopmental disorder with language dela...,2025,L190P,P
120,AGO1,Q9UL18,VAR_088409,p.Leu190Arg,LP/P,rs1645264815,Neurodevelopmental disorder with language dela...,2025,L190R,P


In [27]:
data25_2.to_csv('../data/humsavar/humsavar_20212025.csv', index=0)

We add more information to the dataset by retrieving .json files from Uniprot.

MANE-Select (Matched Annotation from NCBI and EMBL-EBI) provides a single annotated transcript per gene, thus giving consistency between RefSeq and Ensembl. The addition of MANE-Select data helps standardize variant annotations. 

With this we aim to link variants to a reliable reference transcript.

In [28]:
def extract_mane_select_info(humsavar_df, uniprot_id_column="Swiss-Prot AC", 
                           cache_dir="uniprot_cache", 
                           rate_limit_delay=0.1,
                           batch_size=100):
    """
    Extracts MANE-Select information for each UniProt ID with caching and rate limiting.
    
    Args:
        humsavar_df (pd.DataFrame): DataFrame containing UniProt IDs
        uniprot_id_column (str): Name of the column containing UniProt IDs
        cache_dir (str): Directory to store cached responses
        rate_limit_delay (float): Delay between API calls in seconds
        batch_size (int): Number of proteins to process before saving interim results
    
    Returns:
        pd.DataFrame: DataFrame with MANE-Select information
    """

    cache_path = Path(cache_dir)
    cache_path.mkdir(parents=True, exist_ok=True)
    log_file = cache_path / f"extraction_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    
    def log_message(message):
        """Write message to log file and print it"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_message = f"[{timestamp}] {message}"
        print(log_message)
        with open(log_file, "a") as f:
            f.write(log_message + "\n")

    def get_cached_response(uniprot_id):
        """Get cached response for a UniProt ID"""
        cache_file = cache_path / f"{uniprot_id}.json"
        if cache_file.exists():
            try:
                with open(cache_file, "r") as f:
                    return json.load(f)
            except json.JSONDecodeError:
                return None
        return None
    
    def cache_response(uniprot_id, data):
        """Cache response for a UniProt ID"""
        cache_file = cache_path / f"{uniprot_id}.json"
        with open(cache_file, "w") as f:
            json.dump(data, f)
    
    def process_uniprot_id(uniprot_id):
        """Process a single UniProt ID"""
        cached_data = get_cached_response(uniprot_id)
        if cached_data is not None:
            data = cached_data
            log_message(f"Using cached data for {uniprot_id}")
        else:
            try:
                # Fetch JSON data from UniProt API
                json_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"
                response = requests.get(json_url)
                response.raise_for_status()
                data = response.json()
                # Cache the response
                cache_response(uniprot_id, data)
                log_message(f"Fetched and cached data for {uniprot_id}")
                # Rate limiting
                time.sleep(rate_limit_delay)
            except Exception as e:
                log_message(f"Error fetching data for {uniprot_id}: {str(e)}")
                return {
                    'Swiss-Prot AC': uniprot_id,
                    'MANE-Select ID': None,
                    'Ensembl ID': None,
                    'Protein ID': None,
                    'RefSeq Nucleotide ID': None,
                    'RefSeq Protein ID': None,
                    'Error': str(e)
                }
        
        try:
            # Search for MANE-Select information
            mane_select_info = None
            for entry in data.get('uniProtKBCrossReferences', []):
                if entry.get('database') == 'MANE-Select':
                    mane_select_info = entry
                    break
            
            if mane_select_info:
                mane_id = mane_select_info['id']
                ensembl_id = mane_select_info['id'].split('.')[0]
                protein_id = next(
                    (prop['value'] for prop in mane_select_info['properties'] 
                     if prop['key'] == 'ProteinId'), None)
                refseq_nucleotide_id = next(
                    (prop['value'] for prop in mane_select_info['properties'] 
                     if prop['key'] == 'RefSeqNucleotideId'), None)
                refseq_protein_id = next(
                    (prop['value'] for prop in mane_select_info['properties'] 
                     if prop['key'] == 'RefSeqProteinId'), None)
            else:
                mane_id = ensembl_id = protein_id = refseq_nucleotide_id = refseq_protein_id = None
            
            return {
                'Swiss-Prot AC': uniprot_id,
                'MANE-Select ID': mane_id,
                'Ensembl ID': ensembl_id,
                'Protein ID': protein_id,
                'RefSeq Nucleotide ID': refseq_nucleotide_id,
                'RefSeq Protein ID': refseq_protein_id,
                'Error': None
            }
        except Exception as e:
            log_message(f"Error processing data for {uniprot_id}: {str(e)}")
            return {
                'Swiss-Prot AC': uniprot_id,
                'MANE-Select ID': None,
                'Ensembl ID': None,
                'Protein ID': None,
                'RefSeq Nucleotide ID': None,
                'RefSeq Protein ID': None,
                'Error': str(e)
            }

    results = []
    total_ids = len(humsavar_df[uniprot_id_column].unique())
    log_message(f"Starting processing of {total_ids} Uniprot IDs")
    
    for i, uniprot_id in enumerate(humsavar_df[uniprot_id_column].unique(), 1):
        result = process_uniprot_id(uniprot_id)
        results.append(result)

        if i % batch_size == 0:
            interim_df = pd.DataFrame(results)
            interim_file = cache_path / f"interim_results_{i}.csv"
            interim_df.to_csv(interim_file, index=False)
            log_message(f"Processed {i}/{total_ids} IDs. Saved results to {interim_file}")

    mane_select_df = pd.DataFrame(results)

    final_file = cache_path/"final_results.csv"
    mane_select_df.to_csv(final_file, index=False)
    log_message(f"Final results saved to {final_file}")
    return mane_select_df

In [29]:
data25_2 = pd.read_csv('../data/humsavar/humsavar_20212025.csv')

Retrieve and add MANE-Select transcript mappings to the dataset.

In [30]:
mane_results = extract_mane_select_info(
    data25_2,
    cache_dir="uniprot_cache",  # store API responses
    rate_limit_delay=0.1,       # delay between API calls in seconds
    batch_size=100              # save results every 100 proteins
)

[2025-03-18 16:45:29] Starting processing of 1257 Uniprot IDs
[2025-03-18 16:45:29] Using cached data for P49588
[2025-03-18 16:45:29] Using cached data for P78363
[2025-03-18 16:45:29] Using cached data for Q9NP58
[2025-03-18 16:45:29] Using cached data for O14678
[2025-03-18 16:45:29] Using cached data for Q96SE0
[2025-03-18 16:45:29] Using cached data for O95870
[2025-03-18 16:45:29] Using cached data for A6QL63
[2025-03-18 16:45:29] Using cached data for P25106
[2025-03-18 16:45:29] Using cached data for A6NK06
[2025-03-18 16:45:29] Using cached data for Q3I5F7
[2025-03-18 16:45:29] Using cached data for Q9ULC5
[2025-03-18 16:45:29] Using cached data for P68133
[2025-03-18 16:45:29] Using cached data for P62736
[2025-03-18 16:45:29] Using cached data for P60709
[2025-03-18 16:45:29] Using cached data for P63267
[2025-03-18 16:45:29] Using cached data for Q8TC94
[2025-03-18 16:45:29] Using cached data for P35609
[2025-03-18 16:45:29] Using cached data for O60266
[2025-03-18 16:45:29

In [31]:
mane_results

,Swiss-Prot AC,MANE-Select ID,Ensembl ID,Protein ID,RefSeq Nucleotide ID,RefSeq Protein ID,Error
0,P49588,ENST00000261772.13,ENST00000261772,ENSP00000261772.8,NM_001605.3,NP_001596.2,None
1,P78363,ENST00000370225.4,ENST00000370225,ENSP00000359245.3,NM_000350.3,NP_000341.2,None
2,Q9NP58,ENST00000265316.9,ENST00000265316,ENSP00000265316.3,NM_005689.4,NP_005680.1,None
3,O14678,ENST00000356924.9,ENST00000356924,ENSP00000349396.4,NM_005050.4,NP_005041.1,None
4,Q96SE0,ENST00000316470.9,ENST00000316470,ENSP00000326491.4,NM_032604.4,NP_115993.3,None
...,...,...,...,...,...,...,...
1252,Q96K58,ENST00000300849.5,ENST00000300849,ENSP00000300849.4,NM_024706.5,NP_078982.3,None
1253,P17019,ENST00000356929.3,ENST00000356929,ENSP00000349401.2,NM_021269.3,NP_067092.2,None
1254,O60290,ENST00000223210.5,ENST00000223210,ENSP00000223210.4,NM_001099220.3,NP_001092690.1,None
1255,Q9BRT8,ENST00000356521.9,ENST00000356521,ENSP00000348915.4,NM_018491.5,NP_060961.3,None


In [32]:
# check if there are any errors
errors = mane_results[mane_results['Error'].notna()]

if len(errors) > 0:
    print("\nEntries with errors:")
    print(errors[['Swiss-Prot AC', 'Error']].head())

Now we can combine the retrieved transcript mappings with the main dataset.

In [33]:
merged_df = data25_2.merge(mane_results, on="Swiss-Prot AC", how="left")
merged_df.rename(columns={'Swiss-Prot AC':'Uniprot'}, inplace=True)

Also, important to drop duplicate rows again!

In [34]:
merged_df=merged_df.drop_duplicates().reset_index(drop=True)
len(merged_df)

3824

In [35]:
merged_df.head()

,Main gene name,Uniprot,FTId,AA change,Variant category,dbSNP,Disease name,Source,Variant,BinaryClinicalSignificance,MANE-Select ID,Ensembl ID,Protein ID,RefSeq Nucleotide ID,RefSeq Protein ID,Error
0,AARS1,P49588,VAR_089576,p.Arg326Trp,LP/P,-,"Charcot-Marie-Tooth disease, axonal, 2N (CMT2N...",2025,R326W,P,ENST00000261772.13,ENST00000261772,ENSP00000261772.8,NM_001605.3,NP_001596.2,None
1,AARS1,P49588,VAR_089577,p.Thr606Ile,LP/P,-,"Leukoencephalopathy, hereditary diffuse, with ...",2025,T606I,P,ENST00000261772.13,ENST00000261772,ENSP00000261772.8,NM_001605.3,NP_001596.2,None
2,AARS1,P49588,VAR_089578,p.Ser698Phe,LP/P,-,"Charcot-Marie-Tooth disease, axonal, 2N (CMT2N...",2025,S698F,P,ENST00000261772.13,ENST00000261772,ENSP00000261772.8,NM_001605.3,NP_001596.2,None
3,ABCA4,P78363,VAR_084908,p.Asp1102Tyr,LB/B,rs138641544,NaN,2025,D1102Y,B,ENST00000370225.4,ENST00000370225,ENSP00000359245.3,NM_000350.3,NP_000341.2,None
4,ABCA4,P78363,VAR_084916,p.Gly1203Asp,LP/P,-,Stargardt disease 1 (STGD1) [MIM:248200],2025,G1203D,P,ENST00000370225.4,ENST00000370225,ENSP00000359245.3,NM_000350.3,NP_000341.2,None


In [46]:
merged_df.to_csv('../data/humsavar/humsavar_20212025_v2.csv',index=0)

We are ready to prepare input for the VEP tool and obtain predictions.

But there is a problem: Humsavar data is in Aa format, so there is no chromosome, genomic coordinate info etc (needed for VEPs).

Possible solutions to this:
- __Option 1__: Retrieve only those variants with dbSNP ID number
- __Option 2__: Retrieve nucleotide change information from REVEL downloads (_Fail. Revel coverage is not wide enough_)
- __Option 3__: Try generating VCF file for all those proteins from the transcript/exon information (_Time consuming_)
- __Option 4__: Select 10 predictors and get predictions separately from each tool (_Maybe/Some tools still require genomic coodinate format_)


### __Option 1__: Extract variants with a known dbSNP ID for easier mapping to genomic coordinates.

In [47]:
merged_df = pd.read_csv('../data/humsavar/humsavar_20212025_v2.csv')

In [48]:
merged_df.dbSNP.nunique()

2577

In [49]:
# keep these in mind to check after
merged_df[merged_df.duplicated(subset=['dbSNP'], keep=False)].dbSNP.unique()

array(['-', 'rs1645264815', 'rs2153228682', 'rs1705222655', 'rs752450983',
       'rs1599011050', 'rs1949512456', 'rs28642966', 'rs201552310',
       'rs200005406', 'rs2072648', 'rs1057517926', 'rs1757708758',
       'rs77834747', 'rs381427', 'rs421016', 'rs121908310', 'rs77933015',
       'rs121908308', 'rs782199122', 'rs1191455921', 'rs2071312',
       'rs104894264', 'rs7480563', 'rs7126405', 'rs2293232', 'rs2246901',
       'rs200291894', 'rs571714796', 'rs7255187', 'rs1684813071',
       'rs1554297905', 'rs1838076782', 'rs141269120', 'rs1057149',
       'rs1385657144', 'rs1965499910'], dtype=object)

We remove those entries without dbSNP annotations.

In [50]:
tmp=merged_df.drop_duplicates(subset='dbSNP', keep='first')
dbsnplist=list(tmp[tmp.dbSNP!='-'].dbSNP.values)

And analyze the dataset coverage.

In [51]:
print(len(merged_df))  # Total variants  
print(len(dbsnplist))  # Unique dbSNP IDs  
print(len(merged_df[merged_df.dbSNP!='-']))   # Variants with dbSNP IDs  
print(merged_df[merged_df.dbSNP!='-']['Uniprot'].nunique())  # Unique proteins with dbSNP mapped variants  

3824
2576
2613
976


In [54]:
with open('../data/humsavar/humsavar_rsIDs.txt', 'w') as f:
    for db in dbsnplist:
        f.write(f"{db}" + "\n")

Then, identify entries lacking RefSeq Protein ID.

In [53]:
merged_df[merged_df['RefSeq Protein ID'].isna()]

,Main gene name,Uniprot,FTId,AA change,Variant category,dbSNP,Disease name,Source,Variant,BinaryClinicalSignificance,MANE-Select ID,Ensembl ID,Protein ID,RefSeq Nucleotide ID,RefSeq Protein ID,Error
62,ADGRF2P,Q8IZF7,VAR_024473,p.Gln148Arg,LB/B,rs6907125,NaN,2025,Q148R,B,NaN,NaN,NaN,NaN,NaN,NaN
63,ADGRF2P,Q8IZF7,VAR_024474,p.Ile467Val,LB/B,rs9381594,NaN,2025,I467V,B,NaN,NaN,NaN,NaN,NaN,NaN
152,AHI1-DT,P0C7V0,VAR_056807,p.Ala112Glu,LB/B,rs13197384,NaN,2025,A112E,B,NaN,NaN,NaN,NaN,NaN,NaN
156,AKR1C8,Q5T2L2,VAR_032355,p.Arg50His,LB/B,rs7097295,NaN,2025,R50H,B,NaN,NaN,NaN,NaN,NaN,NaN
207,ANKRD26P1,Q6NSI1,VAR_040003,p.Lys265Thr,LB/B,rs1436436,NaN,2025,K265T,B,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3746,WWC3,Q9ULE0,VAR_036970,p.Tyr717Cys,LB/B,rs36076296,NaN,2025,Y717C,B,NaN,NaN,NaN,NaN,NaN,NaN
3747,WWC3,Q9ULE0,VAR_062109,p.Pro955Leu,LB/B,rs55787431,NaN,2025,P955L,B,NaN,NaN,NaN,NaN,NaN,NaN
3765,YWHAH-AS1,Q9Y442,VAR_050934,p.His11Leu,LB/B,rs1984388,NaN,2025,H11L,B,NaN,NaN,NaN,NaN,NaN,NaN
3790,ZNF22-AS1,Q5T742,VAR_047102,p.Ile63Asn,LB/B,rs12269028,NaN,2025,I63N,B,NaN,NaN,NaN,NaN,NaN,NaN


### Read VEP output (predictions added!)

In [ ]:
VEP_output = pd.read_csv('../data/humsavar/vep_rsout.txt', sep='\t')
VEP_output.head()

In [ ]:
VEP_output['#Uploaded_variation'].nunique()

In [ ]:
VEP_output.columns()

In [ ]:
df = VEP.clean_vep_data(VEP_output)
df.head()

In [ ]:
print(len(df))

Now, we merge the ouput with the previous humsavar dataset (to just retain those variants with dbSNP annotations.)

In [ ]:
len(merged_df)

In [ ]:
for i in dbsnplist:
    if i not in VEP_output['#Uploaded_variation'].unique():
        print(i)

In [ ]:
merged_df[merged_df.dbSNP.isin(['rs1141815','rs11541954'])]

In [ ]:
for col in merged_df.columns:
    if col in df.columns:
        print(col)

In [ ]:
def adjustments_before_merging_hum(humsavar,df):
    
    # adjustments for humsavar dataset
    hum_to_merge = humsavar.copy()
    hum_to_merge = hum_to_merge[hum_to_merge.dbSNP!='-'].reset_index(drop=True)
    print("Variants with dbSNP IDs: ", len(hum_to_merge))


    # adjustments for vep output
    df_to_merge=df.copy()
    df_to_merge['Chromosome'] = df_to_merge.apply(lambda x: str(x['Location'].split(':')[0]), axis=1)
    df_to_merge.rename(columns={'SYMBOL':'Main gene name'}, inplace=True)
    
    df_to_merge['dbSNP'] = df_to_merge['#Uploaded_variation']
    df_to_merge2=df_to_merge.drop_duplicates(subset=['#Uploaded_variation', 'Location', 'Allele', 'Main gene name', 'Gene','Feature_type',
                  'Protein_position', 'Amino_acids','Chromosome'], keep='first').reset_index(drop=True)
    
    print(df_to_merge2.BIOTYPE.unique())

    df_to_merge3 = df_to_merge2[df_to_merge2.BIOTYPE=='protein_coding']
    df_to_merge3 = df_to_merge3[~df_to_merge3['Amino_acids'].str.contains('\*')]
    df_to_merge3 = df_to_merge3[df_to_merge3['Amino_acids'].str.contains('/')]
    df_to_merge3['variant']=df_to_merge3.apply(lambda x: x['Amino_acids'].split('/')[0] + str(x['Protein_position'])+ x['Amino_acids'].split('/')[1], axis=1)

    df_to_merge3['position'] = pd.to_numeric(df_to_merge3['Protein_position'], errors='coerce')
    df_to_merge3=df_to_merge3[df_to_merge3['position'].notna()].reset_index(drop=True)

    print("?: ", len(df_to_merge), len(df_to_merge2),len(df_to_merge3))
    
    return hum_to_merge, df_to_merge3

In [ ]:
hum_merge, df_merge = adjustments_before_merging_hum(merged_df,df)

In [ ]:
df_merge['#Uploaded_variation'].nunique()

In [ ]:
print(len(hum_merge))
print(df_merge['dbSNP'].nunique())

In [ ]:
2339-2306  # number of variants I expect to loose with this merge

In [ ]:
merge_cols = ['Main gene name','variant','dbSNP']

In [ ]:
hum_merge[merge_cols].head()

In [ ]:
df_merge[merge_cols].head()

In [ ]:
hum_merged=hum_merge.merge(df_merge, on=merge_cols, how='left')

print(len(hum_merged))
hum_merged.head()

In [ ]:
# the same rs id might not match with the vep results in terms of variant...

collect_rs=[]
for rs in df_merge['#Uploaded_variation'].unique():
    if rs not in hum_merged['#Uploaded_variation'].unique():
        collect_rs.append(rs)
len(collect_rs)

In [ ]:
#same rsid matching different variant
df_merge[df_merge['#Uploaded_variation']=='rs2145977887'][['#Uploaded_variation','dbSNP','Main gene name','variant']]

In [ ]:
# and in humsavar mapping to a completely different variant
hum_merge[hum_merge['dbSNP']=='rs2145977887'][['dbSNP','Main gene name','variant']]

In [ ]:
hum_merged['#Uploaded_variation'].nunique()

In [ ]:
hum_merged2=hum_merged.drop_duplicates(subset=['Main gene name','Uniprot','AA change','Variant category', 
                                               'Disease name','Source','variant','BinaryClinicalSignificance',
                                               'MANE-Select ID','Ensembl ID','Protein ID','RefSeq Nucleotide ID',
                                               'RefSeq Protein ID'], 
                                               keep='first')

len(hum_merged2[hum_merged2['#Uploaded_variation'].notna()])

In [ ]:
len(hum_merged2) == len(merged_df[merged_df.dbSNP!='-'])

In [ ]:
def check_coverage(df):
    # check coverage
    cols=[i for i in df.columns if '_binary' in i]
    
    adding=[]
    for i in cols:  
        tmp = df[df[i].notna()]
        i =i.split('_')[0]
        if i == 'am':
            i = 'AlphaMissense'  
        coverage = round(100 * len(tmp) / len(df), 2) 
        adding.append([i, coverage])
        
    table=pd.DataFrame(adding, columns=['Predictor','Coverage'])
    table=table.sort_values('Coverage',ascending=False).reset_index(drop=True)
    return table

In [ ]:
check_coverage(hum_merged2)

In [ ]:
hum_merged2.BinaryClinicalSignificance.value_counts()

In [ ]:
hum_merged2.to_csv('../data/humsavar/cleaned_Humsavar_dataset_VEP.csv',index=0)

We can compare how both datasets differ in terms of variants.

In [ ]:
humsavar = pd.read_csv('../data/humsavar/cleaned_Humsavar_dataset_VEP.csv')
humsavar.head(3)

In [ ]:
print(humsavar.BinaryClinicalSignificance.value_counts())
print(len(humsavar))
print(humsavar.Uniprot.nunique())
print(humsavar['Main gene name'].nunique())

In [ ]:
clinvar = pd.read_csv('../data/clinvar/cleaned_Humsavar_dataset_VEP.csv', low_memory=False)
clinvar.head(3)

In [ ]:
clinvar.uniprot.nunique()

In [ ]:
print(clinvar.BinaryClinicalSignificance.value_counts())

print(len(clinvar))
print(clinvar.uniprot.nunique())
print(clinvar['GeneSymbol'].nunique())

In [ ]:
collect = []
collect_indexes=[]
for uni in humsavar.Uniprot.unique():
    if uni in clinvar.uniprot.unique():
        for var in humsavar[humsavar.Uniprot ==uni].variant.values:
            if var in clinvar[clinvar.uniprot==uni].variant.values:
                #collect.append([uni,var])
                index_hum = humsavar[(humsavar.Uniprot ==uni) & (humsavar.variant==var)].index[0]
                collect_indexes.append(index_hum)

In [ ]:
len(collect_indexes)

In [ ]:
humsavar_no_clinvar=humsavar[~humsavar.index.isin(collect_indexes)]

In [ ]:
print(humsavar_no_clinvar.BinaryClinicalSignificance.value_counts())

print(len(humsavar_no_clinvar))
print(humsavar_no_clinvar.Uniprot.nunique())
print(humsavar_no_clinvar['Main gene name'].nunique())

In [ ]:
check_coverage(humsavar_no_clinvar)

In [ ]:
humsavar_no_clinvar=humsavar_no_clinvar.reset_index(drop=True)
humsavar_no_clinvar.to_csv('../data/humsavar/humsavar_noclinvar_VEP.csv', index=0)